In [6]:
# imports 
from pydub import AudioSegment
from google.cloud import speech
from huggingsound import SpeechRecognitionModel
import os
import io
import pandas as pd


In [9]:
#import tsv with data
text_df = pd.read_csv('../raw_data/commonvoicedataset/validated.tsv', sep='\t', header=0, low_memory=False)

In [17]:
#check df
text_df.head(2)

,client_id,path,sentence,up_votes,down_votes,age,gender,accents,locale,segment
0,09e8441e62e3c8da70b667874fa75e96731f6e43e359a1...,common_voice_pt_27283586.mp3,"Se esta primeira condição for satisfeita, é se...",2,0,NaN,NaN,NaN,pt,NaN
1,12baee49ea5072cfd8392765aeb6d0e518a51a23224aa8...,common_voice_pt_25643625.mp3,Nós temos tempo suficiente.,2,1,NaN,NaN,NaN,pt,NaN


In [26]:
# create dict with samples file name and sentence
sample = {}
for p, s in text_df[['path','sentence']].iloc[0:100].itertuples(index=False):
    sample[p] = s
    

In [140]:
#convert files to wav
for file in sample:
    sound = AudioSegment.from_mp3(f"../raw_data/commonvoicedataset/clips/{file}")
    sound = sound.set_frame_rate(16000)
    sound.export(f"../raw_data/{file[:-4]}.wav", format="wav")

E0406 00:51:48.927431400   31247 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0406 00:51:48.983856800   31247 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0406 00:51:49.072774000   31247 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0406 00:51:49.129150600   31247 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0406 00:51:49.218088900   31247 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0406 00:51:49.274207200   31247 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0406 00:51:49.371812500   31247 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0406 00:51:49.427840000   31247 fork_pos

E0406 00:51:53.530400500   31247 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0406 00:51:53.631847500   31247 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0406 00:51:53.689798200   31247 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0406 00:51:53.778261700   31247 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0406 00:51:53.834522200   31247 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0406 00:51:53.923103000   31247 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0406 00:51:54.010027100   31247 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0406 00:51:54.172774400   31247 fork_pos

E0406 00:51:58.517703900   31247 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0406 00:51:58.573424100   31247 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0406 00:51:58.663505900   31247 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0406 00:51:58.719282800   31247 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0406 00:51:58.809855200   31247 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0406 00:51:58.865558100   31247 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0406 00:51:58.955430400   31247 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0406 00:51:59.011270100   31247 fork_pos

E0406 00:52:03.127773300   31247 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0406 00:52:03.220988400   31247 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0406 00:52:03.276983600   31247 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0406 00:52:03.364336100   31247 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0406 00:52:03.420515000   31247 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0406 00:52:03.505635900   31247 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0406 00:52:03.564070600   31247 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0406 00:52:03.670353100   31247 fork_pos

In [84]:
# save audios names
audio_names = list(sample.keys())

In [120]:
#create paths to audio
audio_paths = []
for name in audio_names:
    audio_paths.append(f"../raw_data/{name[:-4]}.wav")
# remove other raw data
#audio_paths.remove('../raw_data/commonvoicedataset')

In [122]:
#use Wav2Vec model from huggingsound
model = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-portuguese")

transcriptions = model.transcribe(audio_paths)

04/06/2022 00:27:16 - INFO - huggingsound.speech_recognition.model - Loading model...


100%|█████████████████████████| 100/100 [02:01<00:00,  1.21s/it]


In [123]:
#create list of transcriptions results
transcriptions_list = []
for transcript in transcriptions:
    transcriptions_list.append(transcript['transcription'])

In [125]:
#create sample dataframe with filename and sentence
sample_df = pd.DataFrame(list(sample.items()), columns=['file_name', 'sentence'])

In [127]:
#add transcribed sentence list
sample_df['transc_sentence'] = transcriptions_list

In [129]:
#check sample df
sample_df.head()

,file_name,sentence,transc_sentence
0,common_voice_pt_27283586.mp3,"Se esta primeira condição for satisfeita, é se...",se esta família condição for satisfeita é sens...
1,common_voice_pt_25643625.mp3,Nós temos tempo suficiente.,nós temos tempos sob istlan
2,common_voice_pt_27023139.mp3,"Bem, digitalizar Don Quixote é um passo para l...",bem ditalizar do ke shot é oun passo para leva...
3,common_voice_pt_30386095.mp3,Ariranha,aria venho
4,common_voice_pt_24001131.mp3,Ele é advogado do comando vermelho.,ele é advogado do comando vermelho


In [130]:
#check true sentences vs transcribed sentences 
for i, j in sample_df.iterrows():
    print(i, j)

0 file_name                               common_voice_pt_27283586.mp3
sentence           Se esta primeira condição for satisfeita, é se...
transc_sentence    se esta família condição for satisfeita é sens...
Name: 0, dtype: object
1 file_name          common_voice_pt_25643625.mp3
sentence            Nós temos tempo suficiente.
transc_sentence     nós temos tempos sob istlan
Name: 1, dtype: object
2 file_name                               common_voice_pt_27023139.mp3
sentence           Bem, digitalizar Don Quixote é um passo para l...
transc_sentence    bem ditalizar do ke shot é oun passo para leva...
Name: 2, dtype: object
3 file_name          common_voice_pt_30386095.mp3
sentence                               Ariranha
transc_sentence                      aria venho
Name: 3, dtype: object
4 file_name                 common_voice_pt_24001131.mp3
sentence           Ele é advogado do comando vermelho.
transc_sentence     ele é advogado do comando vermelho
Name: 4, dtype: object
5 file_n

In [145]:
#test another model

from transformers import AutoTokenizer, Wav2Vec2ForCTC
  
tokenizer = AutoTokenizer.from_pretrained("Edresson/wav2vec2-large-xlsr-coraa-portuguese")

model_2 = SpeechRecognitionModel("Edresson/wav2vec2-large-xlsr-coraa-portuguese")

/home/alexandre/.pyenv/versions/3.8.12/envs/hear2me/lib/python3.8/site-packages/transformers/configuration_utils.py:356: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


04/06/2022 01:02:56 - INFO - huggingsound.speech_recognition.model - Loading model...


Downloading: 100%|██████████████| 214/214 [00:00<00:00, 231kB/s]


In [146]:
transcriptions_2 = model_2.transcribe(audio_paths)

100%|█████████████████████████| 100/100 [01:53<00:00,  1.13s/it]


In [149]:
#create list of transcriptions results
transcriptions_list_2 = []
for transcript in transcriptions_2:
    transcriptions_list_2.append(transcript['transcription'])
    
#create sample dataframe with filename and sentence
sample_df_2 = pd.DataFrame(list(sample.items()), columns=['file_name', 'sentence'])

#add transcribed sentence list
sample_df_2['transc_sentence'] = transcriptions_list_2

#check true sentences vs transcribed sentences 
for i, j in sample_df_2.iterrows():
    print(i, j)

0 file_name                               common_voice_pt_27283586.mp3
sentence           Se esta primeira condição for satisfeita, é se...
transc_sentence    se esta primília condição for satisfeita é sen...
Name: 0, dtype: object
1 file_name          common_voice_pt_25643625.mp3
sentence            Nós temos tempo suficiente.
transc_sentence        noss tes tempos estunone
Name: 1, dtype: object
2 file_name                               common_voice_pt_27023139.mp3
sentence           Bem, digitalizar Don Quixote é um passo para l...
transc_sentence    bem diritalizar dom que shat é um passo para l...
Name: 2, dtype: object
3 file_name          common_voice_pt_30386095.mp3
sentence                               Ariranha
transc_sentence                     aris aí não
Name: 3, dtype: object
4 file_name                 common_voice_pt_24001131.mp3
sentence           Ele é advogado do comando vermelho.
transc_sentence     ele é advogado do comando vermelho
Name: 4, dtype: object
5 file_n

In [136]:
#test google speech to text

# Creates google client
client = speech.SpeechClient()

# Full path of the audio file, Replace with your file name
file_name = "../raw_data/common_voice_pt_27283586.wav"

#Loads the audio file into memory
with io.open(file_name, "rb") as audio_file:
    content = audio_file.read()
    audio = speech.RecognitionAudio(content=content)

config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    audio_channel_count=2,
    language_code="pt-BR",
)

# Sends the request to google to transcribe the audio
response = client.recognize(request={"config": config, "audio": audio})

# Reads the response
for result in response.results:
    print("Transcript: {}".format(result.alternatives[0].transcript))

PermissionDenied: 403 Cloud Speech-to-Text API has not been used in project 288422971745 before or it is disabled. Enable it by visiting https://console.developers.google.com/apis/api/speech.googleapis.com/overview?project=288422971745 then retry. If you enabled this API recently, wait a few minutes for the action to propagate to our systems and retry. [links {
  description: "Google developers console API activation"
  url: "https://console.developers.google.com/apis/api/speech.googleapis.com/overview?project=288422971745"
}
, reason: "SERVICE_DISABLED"
domain: "googleapis.com"
metadata {
  key: "consumer"
  value: "projects/288422971745"
}
metadata {
  key: "service"
  value: "speech.googleapis.com"
}
]